In [1]:
import pandas as pd

df = pd.read_csv('new_training_data_v7_2019_2020_feb_13_20201.csv').iloc[:,1:]
df

,Datetime,AIL,trend,hour_of_day,just_date,day,hour_x_day,sin.day,cos.day,sin.hour,...,temp_ftmcmry,wind_ftmcmry,temp_lthbrg,wind_lthbrg,temp_mdcnht,wind_mdcnht,temp_rddr,wind_rddr,temp_slvlk,wind_slvlk
0,2019-01-01 00:00:00,10008.0,1,0,2019-01-01,1,0,0.017213,0.999852,0.000000,...,-18.1,11,-11.8,NaN,-12.4,NaN,-15.2,11.0,-14.4,8.0
1,2019-01-01 01:00:00,9868.0,2,1,2019-01-01,1,1,0.017213,0.999852,0.258819,...,-17.0,9,-10.7,NaN,-12.5,NaN,-13.3,5.0,-13.6,5.0
2,2019-01-01 02:00:00,9736.0,3,2,2019-01-01,1,2,0.017213,0.999852,0.500000,...,-15.2,8,-8.5,NaN,-11.8,NaN,-11.8,13.0,-12.9,5.0
3,2019-01-01 03:00:00,9597.0,4,3,2019-01-01,1,3,0.017213,0.999852,0.707107,...,-13.2,12,-6.7,NaN,-10.6,NaN,-10.2,14.0,-14.1,5.0
4,2019-01-01 04:00:00,9530.0,5,4,2019-01-01,1,4,0.017213,0.999852,0.866025,...,-11.5,9,-3.6,NaN,-8.1,NaN,-8.8,17.0,-12.2,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18505,2021-02-10 19:00:00,11485.0,18524,19,2021-02-10,41,779,0.648630,0.761104,-0.965926,...,-29.0,20,-27.0,20.0,-27.0,20.0,-30.0,20.0,-26.0,10.0
18506,2021-02-10 20:00:00,11364.0,18525,20,2021-02-10,41,820,0.648630,0.761104,-0.866025,...,-30.0,20,-28.0,20.0,-28.0,20.0,-31.0,20.0,-27.0,10.0
18507,2021-02-10 21:00:00,11195.0,18526,21,2021-02-10,41,861,0.648630,0.761104,-0.707107,...,-31.0,10,-31.0,10.0,-29.0,10.0,-32.0,10.0,-28.0,10.0
18508,2021-02-10 22:00:00,10961.0,18527,22,2021-02-10,41,902,0.648630,0.761104,-0.500000,...,-32.0,10,-31.0,10.0,-29.0,10.0,-33.0,10.0,-29.0,10.0


In [2]:
df.columns

Index(['Datetime', 'AIL', 'trend', 'hour_of_day', 'just_date', 'day',
       'hour_x_day', 'sin.day', 'cos.day', 'sin.hour', 'cos.hour', 'sin.trend',
       'cos.trend', 'weekend', 'month', 'year', 'sunlight_avaialbility',
       'AIL_previous_hour', 'AIL_24h_lagged', 'AIL_2day_lagged',
       'AIL_3day_lagged', 'AIL_4day_lagged', 'AIL_5day_lagged',
       'AIL_6day_lagged', 'AIL_oneweek_lagged', 'temp_calgary', 'wind_calgary',
       'temp_edmonton', 'wind_edmonton', 'temp_ftmcmry', 'wind_ftmcmry',
       'temp_lthbrg', 'wind_lthbrg', 'temp_mdcnht', 'wind_mdcnht', 'temp_rddr',
       'wind_rddr', 'temp_slvlk', 'wind_slvlk'],
      dtype='object')

In [3]:
import numpy as np

from xgboost import XGBRegressor
from sklearn.model_selection import TimeSeriesSplit
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
import pickle
import hyperopt
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error


#df = df.interpolate(method='nearest').ffill().bfill()
#df.isnull().sum()




#file_name   =  "feature_v2.pkl"
#open_file   =  open(file_name, "rb")
#loaded_list =  pickle.load(open_file)
#open_file.close()

df_hypopt       = df
split_date      = '2020-06-01 00:00:00'
df_hypopt_train = df_hypopt.loc[df_hypopt.Datetime < split_date].copy()
df_hypopt_test  = df_hypopt.loc[df_hypopt.Datetime >= split_date].copy()

y_train         = df_hypopt_train['AIL'].values
y_test          = df_hypopt_test['AIL'].values
X_train         = df_hypopt_train.drop(['Datetime', 'AIL', 'just_date', 'trend', 'sin.trend',
                                        'cos.trend','sin.day', 'cos.day', 'sin.hour', 'cos.hour',
                                        'wind_calgary', 'wind_edmonton', 'temp_ftmcmry', 'wind_ftmcmry',
                                         'temp_lthbrg', 'wind_lthbrg', 'temp_mdcnht', 'wind_mdcnht', 
                                        'temp_rddr','wind_rddr', 'temp_slvlk', 'wind_slvlk'], axis = 1).values

X_test          = df_hypopt_test.drop (['Datetime', 'AIL', 'just_date', 'trend', 'sin.trend',
                                         'cos.trend','sin.day', 'cos.day', 'sin.hour', 'cos.hour',
                                        'wind_calgary','wind_edmonton', 'temp_ftmcmry', 'wind_ftmcmry',
                                         'temp_lthbrg', 'wind_lthbrg', 'temp_mdcnht', 'wind_mdcnht', 
                                        'temp_rddr','wind_rddr', 'temp_slvlk', 'wind_slvlk'], axis = 1).values



space={ 'max_depth'        : hp.quniform("max_depth", 2, 12, 1),
        'gamma'            : hp.uniform ('gamma', 0,1),
        'reg_alpha'        : hp.quniform('reg_alpha', 0,2,1),
        'reg_lambda'       : hp.uniform('reg_lambda', 0,1),
        'colsample_bytree' : hp.uniform('colsample_bytree', 0.5,1),
        'min_child_weight' : hp.quniform('min_child_weight', 1, 7, 1),
        'n_estimators'     : hp.uniform('n_estimators', 10, 300)
    }


#def mape(pred, true):
#    return np.abs(np.asarray(pred) - np.asarray(true)) / np.maximum(np.abs(np.assarry(true))

delta = 10 #huber-delta

def optimize (space):
    model=XGBRegressor(
                       n_estimators     = int(space['n_estimators']), 
                       max_depth        = int(space['max_depth']),
                       gamma            = space['gamma'],
                       reg_alpha        = int(space['reg_alpha']),
                       min_child_weight = space['min_child_weight'],
                       colsample_bytree = space['colsample_bytree'])
    
    model.fit(X_train, y_train)
    train_loss = np.mean(np.abs((y_train - model.predict(X_train)) / y_train)) * 100
    # summarize performance
    preds     = model.predict(X_test)
    #mae       = mean_absolute_error(y_test, preds)
    #loss = np.where(np.abs(y_test - preds) < delta, 
    #                0.5*((y_test - preds)**2),
    #                delta*np.abs(y_test-preds) - 0.5*(delta**2)
    #               )
    loss = np.mean(np.abs((y_test - preds) / y_test)) * 100
    print('train loss: {}, test loss: {}'.format(train_loss, loss))
    return loss
    
trials = Trials()
result = fmin(
            fn        = optimize,
            space     = space,
            algo      = tpe.suggest,
            max_evals = 100,
            trials    = trials
)

print (result)

train loss: 0.3936346511496772, test loss: 0.5312996313153983
train loss: 0.25245131271261656, test loss: 0.8079168888929069                   
train loss: 0.11889135528341763, test loss: 0.5473135624617612                   
train loss: 0.10515425391361523, test loss: 0.7515181684802513                   
train loss: 0.4674760874545175, test loss: 0.8157116094736137                    
train loss: 0.08077335767410031, test loss: 0.5466716889724497                   
train loss: 0.21388734543484356, test loss: 0.5150398700774598                   
train loss: 0.16499392080109473, test loss: 0.5677777942000803                   
train loss: 0.5261921857477716, test loss: 0.5944940389102646                    
train loss: 0.4891950504240333, test loss: 0.5581636866229243                    
train loss: 0.05503962033318045, test loss: 0.6694194767422248                    
train loss: 0.10159519147949932, test loss: 0.5371787330529598                    
train loss: 0.4938098085175524, te

train loss: 0.34072699801953366, test loss: 0.5060650863387863                    
100%|██████████| 100/100 [01:42<00:00,  1.03s/trial, best loss: 0.4999015995906774]
{'colsample_bytree': 0.9685405801358924, 'gamma': 0.5534257429319893, 'max_depth': 5.0, 'min_child_weight': 7.0, 'n_estimators': 178.66950947414804, 'reg_alpha': 1.0, 'reg_lambda': 0.48566044877632586}


In [4]:
import numpy as np
import pandas as pd
from xgboost import XGBRegressor
from datetime import datetime
import joblib
from joblib import dump

column_list = ['hour_of_day',  'day',
       'hour_x_day',  'weekend', 'month', 'year', 'sunlight_avaialbility',
       'AIL_previous_hour', 'AIL_24h_lagged', 'AIL_2day_lagged',
       'AIL_3day_lagged', 'AIL_4day_lagged', 'AIL_5day_lagged',
       'AIL_6day_lagged', 'AIL_oneweek_lagged', 'temp_calgary',
       'temp_edmonton' ]

df_final = pd.DataFrame(data = df)
df_final = df_final.interpolate(method='nearest').ffill().bfill()

y_train = pd.DataFrame(df, columns = ['AIL'])
y_train = y_train.values
X_train = pd.DataFrame(df, columns = column_list)
X_train = X_train.values


'''
{'colsample_bytree': 0.9685405801358924, 'gamma': 0.5534257429319893,
'max_depth': 5.0, 'min_child_weight': 7.0, 
'n_estimators': 178.66950947414804, 'reg_alpha': 1.0,
'reg_lambda': 0.48566044877632586}
'''
model   = XGBRegressor(colsample_bytree = 0.9685405801358924, 
                       gamma            = 0.5589842777690632, 
                       max_depth        = 5, 
                       min_child_weight = 7, 
                       n_estimators     = 179, 
                       reg_alpha        = 1, 
                       reg_lambda       = 0.48566044877632)

model.fit(X_train,y_train)
preds = model.predict(X_train)
print(np.mean(np.abs((y_train - preds) / y_train)) * 100)

# save model to file
model_name = 'model_generated_on_{}_{}_{}_v8_non_holiday_mape_loss_only_temp.joblib.dat'.format(datetime.now().day, datetime.now().month, datetime.now().year)
joblib.dump(model, filename = model_name)

9.950782755081008


['model_generated_on_14_2_2021_v8_non_holiday_mape_loss_only_temp.joblib.dat']